In [2]:
#!pip install -qU langchain_experimental langchain_openai langchain_community langchain ragas chromadb langchain-groq fastembed pypdf openai konlpy sentence-transformers python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.5/309.5 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [8]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.0 MB/s eta 0:00:00


In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter, SpacyTextSplitter, NLTKTextSplitter, split_text_on_tokens, SentenceTransformersTokenTextSplitter, LatexTextSplitter
from langchain.text_splitter import PythonCodeTextSplitter, KonlpyTextSplitter, ElementType

from langchain.document_loaders import PyPDFLoader
import torch
import re
from docx import Document as worddoc

from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.documents.base import Document

import PyPDF2
import pandas as pd
from datasets import Dataset

In [36]:


class TextChunker:
    def __init__(self):
        pass

    def extract_text_from_pdf(self, path = "/"):
        text = ""
        loader = PyPDFLoader(path)
        documents = loader.load()
        for doc in documents:
            text = text + "\n" + str(doc.page_content)
        metadata = self.get_pdf_metadata(path)
        return text,documents, metadata

    def char_count_chunking_with_overlap(self, text, chunk_size=200, chunk_overlap=50, splitter_type = "CharacterTextSplitter"):
        # Instantiate the CharacterTextSplitter class

        text_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

        if splitter_type ==  "RecursiveCharacterTextSplitter":# Instantiate the RecursiveCharacterTextSplitter class
           text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        elif splitter_type == "SpacyTextSplitter":
           text_splitter = SpacyTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

        # Create documents using the text splitter
        docs =[]
        chunks = []
        if isinstance(text, str):
          docs = text_splitter.create_documents([text])
          chunks = [doc.page_content for doc in docs]
        elif isinstance(text, list):
          try:
            docs = text_splitter.create_documents(text)
            chunks = [doc.page_content for doc in docs]
          except:
            chunks = text_splitter.split_documents(text)
        return chunks

    #Nonoverlap
    def char_count_chunking_with_nonoveralp(self, text, chunk_size=200, splitter_type = "CharacterTextSplitter"):
       # Instantiate the CharacterTextSplitter class
        text_splitter = CharacterTextSplitter(chunk_size=chunk_size)

        if splitter_type ==  "RecursiveCharacterTextSplitter":# Instantiate the RecursiveCharacterTextSplitter clas
           text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size)
        elif splitter_type == "SpacyTextSplitter":
           text_splitter = SpacyTextSplitter(chunk_size=chunk_size)
        # Create documents using the text splitter
        docs =[]
        chunks = []
        if isinstance(text, str):
          docs = text_splitter.create_documents([text])
          chunks = [doc.page_content for doc in docs]
        elif isinstance(text, list):
          try:
            docs = text_splitter.create_documents(text)
            chunks = [doc.page_content for doc in docs]
          except:
            chunks = text_splitter.split_documents(text)

        return chunks

    #chunking
    def char_count_chunking_with_custom_delimiter(self, text, chunk_size=200, chunk_overlap=50, delimiter="\n\n",splitter_type = "CharacterTextSplitter"):
        # Instantiate the CharacterTextSplitter class
        text_splitter = CharacterTextSplitter(separator=delimiter, chunk_size=chunk_size, chunk_overlap=chunk_overlap)

        if splitter_type ==  "RecursiveCharacterTextSplitter":# Instantiate the RecursiveCharacterTextSplitter class
           text_splitter = RecursiveCharacterTextSplitter(separator=delimiter, chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        elif splitter_type == "SpacyTextSplitter":
           text_splitter = SpacyTextSplitter(separator=delimiter, chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        elif splitter_type == "NLTKTextSplitter":
           text_splitter = NLTKTextSplitter(separator=delimiter, chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        elif splitter_type == "SpacyTextSplitter":
           text_splitter = SpacyTextSplitter(separator=delimiter, chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        elif splitter_type == "SentenceTransformersTokenTextSplitter":
           text_splitter = SentenceTransformersTokenTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        elif splitter_type == "LatexTextSplitter":
           text_splitter = LatexTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        elif splitter_type == "PythonCodeTextSplitter":
           text_splitter = PythonCodeTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        elif splitter_type == "KonlpyTextSplitter":
           text_splitter = KonlpyTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

        # Create documents using the text splitter
        docs =[]
        chunks = []
        if isinstance(text, str):
          docs = text_splitter.create_documents([text])
          chunks = [doc.page_content for doc in docs]
        else:
          try:
            chunks = text_splitter.split_documents(text)
          except:
            docs = text_splitter.create_documents(text)
            chunks = [doc.page_content for doc in docs]

        return chunks

    #Semantic chunking
    def semantic_section_chunking(self, documents , text_embedding_model_name,  breakpoint_threshold_type = "percentile"):
        embed_model = FastEmbedEmbeddings(model_name = text_embedding_model_name)
        semantic_chunker = SemanticChunker(embed_model, breakpoint_threshold_type=breakpoint_threshold_type)
        try:
          semantic_chunks = semantic_chunker.create_documents([d.page_content for d in documents])
        except:
          semantic_chunks = semantic_chunker.create_documents([d for d in markdown_splits])
        return semantic_chunks

    #Markdown HeaderTextSplitter
    def markdown_header_textsplitter(self, text, headers_to_split_on, strip_headers):
        from langchain.text_splitter import MarkdownHeaderTextSplitter

        markdown_document = text
        markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on,  strip_headers=strip_headers)
        md_header_splits = markdown_splitter.split_text(markdown_document)
        return md_header_splits

    #Markdown TextSplitter
    def markdown_textsplitter(self, text):
        from langchain.text_splitter import MarkdownTextSplitter
        splitter = MarkdownTextSplitter()
        split_text = splitter.split_text(text)
        return split_text

    #ouputs to Doc
    #from docx import Document
    #import re
    def save_as_word_doc(self, split_text, file_path ,heading):
        doc = worddoc()
        doc.add_heading(heading, level=1)
        for i, text_chunk in enumerate(split_text):
            text_chunk= re.sub(r'[\x00-\x1F]+', '', str(text_chunk))
            doc.add_heading(f'Chunk {i + 1}', level=1)
            doc.add_paragraph(text_chunk)
        # Save the document
        doc.save(file_path)
        print("Output saved as doc successfully")

    #Get pdf _metadata
    def get_pdf_metadata(self, pdf_path):
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            metadata = reader.metadata
            return metadata

    #create text dataframe as Langchain Document Schema
    def Create_Hugging_Face_Dataset(self, pdf_path):
        data = {
            'text': [],
            'source': []
        }

        text, documents = self.extract_text_from_pdf(pdf_path)
        metadata = self.get_pdf_metadata(pdf_path)
        source = str(metadata)
        data['text'].append(text)
        data['source'].append(source)
        df = pd.DataFrame(data)
        dataset = Dataset.from_pandas(df)# Convert pandas DataFrame to Hugging Face Dataset
        return dataset, documents

#################################################################################
#Usage
pdf_path =r"/content/drive/MyDrive/chunking/ds (1).pdf"

text_chunker = TextChunker()
text, documents, metadata  = text_chunker.extract_text_from_pdf(path = pdf_path)

In [37]:
#Chunking
markdown_split_text = text_chunker.markdown_textsplitter(text)

SentenceTransformers_chunks = text_chunker.char_count_chunking_with_custom_delimiter(markdown_split_text, chunk_size=200, chunk_overlap=50, splitter_type = "SentenceTransformersTokenTextSplitter")


In [41]:
from sentence_transformers import SentenceTransformer

# Initialize the embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

def generate_embeddings(split_text):
    embeddings = model.encode(split_text, convert_to_tensor=True)
    return embeddings

embeddings = generate_embeddings(SentenceTransformers_chunks)

In [43]:
def attach_metadata(split_texts, embeddings, original_metadata):
    combined_metadata = [{'chunk_number': i, 'original_metadata': original_metadata} for i in range(len(split_texts))]
    embeddings_with_metadata = [{'embedding': embedding, 'metadata': combined_metadata[i]} for i, embedding in enumerate(embeddings)]
    return embeddings_with_metadata

embeddings_with_metadata = attach_metadata(SentenceTransformers_chunks, embeddings, metadata)

In [53]:
import chromadb
from chromadb.config import Settings

# Initialize Chroma client
#client = chromadb.Client(Settings(chroma_db_impl="sqlite", persist_directory="/content/drive/MyDrive/chunking/test/"))
client = chromadb.Client(Settings(persist_directory="/content/drive/MyDrive/CRAG"))#

def store_in_chroma(documents, embeddings, collection_name="crag"):
    collection = client.get_or_create_collection(name=collection_name)
    for doc, embedding in zip(documents, embeddings):
        collection.add(
            embedding=embedding.tolist(),  # Convert numpy array to list
            metadata=doc['metadata'],
            content=doc['content']
        )
    client.persist()
    return collection

collection = store_in_chroma(documents, embeddings_with_metadata)

AttributeError: 'dict' object has no attribute 'tolist'

In [22]:
ds, documents = text_chunker.Create_Hugging_Face_Dataset(pdf_path)
ds

Dataset({
    features: ['text', 'source'],
    num_rows: 1
})

In [26]:
ds[0]['source']

'{\'/Author\': \'\', \'/Title\': \'\', \'/Subject\': \'\', \'/Creator\': \'LaTeX with hyperref package\', \'/Producer\': \'pdfTeX-1.40.19\', \'/Keywords\': \'\', \'/CreationDate\': "D:20190814093040+02\'00\'", \'/ModDate\': "D:20190814093040+02\'00\'", \'/Trapped\': \'/False\', \'/PTEX.Fullbanner\': \'This is pdfTeX, Version 3.14159265-2.6-1.40.19 (TeX Live 2018) kpathsea version 6.3.0\'}'

In [30]:
from langchain.docstore.document import Document as LangchainDocument
from tqdm.notebook import tqdm

RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]}) for doc in tqdm(ds)
]
print(len(RAW_KNOWLEDGE_BASE))
print(RAW_KNOWLEDGE_BASE[0].page_content)
print(RAW_KNOWLEDGE_BASE[0].metadata)

  0%|          | 0/1 [00:00<?, ?it/s]

1

Mathematical Foundations of Data Sciences
Gabriel Peyr´ e
CNRS & DMA
´Ecole Normale Sup´ erieure
gabriel.peyre@ens.fr
https://mathematical-tours.github.io
www.numerical-tours.com
August 14, 2019
2
Chapter 1
Optimal Transport
1.1 Radon Measures
Measures. We will interchangeably the term histogram or probability vector for any element a∈Σnthat
belongs to the probability simplex
Σndef.={
a∈Rn
+;n∑
i=1ai= 1}
.
A discrete measure with weights aand locations x1,...,xn∈X reads
α=n∑
i=1aiδxi (1.1)
whereδxis the Dirac at position x, intuitively a unit of mass which is inﬁnitely concentrated at location
x. Such as measure describes a probability measure if, additionally, a∈Σn, and more generally a positive
measure if each of the “weights” described in vector ais positive itself.
Remark 1 (General measures) .A convenient feature of OT is that it can deal with discrete and continuous
“objects” within the same framework. Such objects only need to be modelled as measures. This corresponds
to the 

In [31]:
markdown_split_text = text_chunker.markdown_textsplitter(text)

SentenceTransformers_chunks = text_chunker.char_count_chunking_with_custom_delimiter(markdown_split_text, chunk_size=200, chunk_overlap=50, splitter_type = "SentenceTransformersTokenTextSplitter")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [33]:
docs_processed = []
for doc in RAW_KNOWLEDGE_BASE:
    print(doc)
    markdown_split_text = text_chunker.markdown_textsplitter([doc])

    SentenceTransformers_chunks = text_chunker.char_count_chunking_with_custom_delimiter(markdown_split_text, chunk_size=200, chunk_overlap=50, splitter_type = "SentenceTransformersTokenTextSplitter")

    docs_processed += SentenceTransformers_chunks

page_content='\nMathematical Foundations of Data Sciences\nGabriel Peyr´ e\nCNRS & DMA\n´Ecole Normale Sup´ erieure\ngabriel.peyre@ens.fr\nhttps://mathematical-tours.github.io\nwww.numerical-tours.com\nAugust 14, 2019\n2\nChapter 1\nOptimal Transport\n1.1 Radon Measures\nMeasures. We will interchangeably the term histogram or probability vector for any element a∈Σnthat\nbelongs to the probability simplex\nΣndef.={\na∈Rn\n+;n∑\ni=1ai= 1}\n.\nA discrete measure with weights aand locations x1,...,xn∈X reads\nα=n∑\ni=1aiδxi (1.1)\nwhereδxis the Dirac at position x, intuitively a unit of mass which is inﬁnitely concentrated at location\nx. Such as measure describes a probability measure if, additionally, a∈Σn, and more generally a positive\nmeasure if each of the “weights” described in vector ais positive itself.\nRemark 1 (General measures) .A convenient feature of OT is that it can deal with discrete and continuous\n“objects” within the same framework. Such objects only need to be modelle

TypeError: expected string or bytes-like object

In [27]:
ds1[0]['source']

'huggingface/hf-endpoints-documentation/blob/main/docs/source/guides/create_endpoint.mdx'

In [5]:
#MarkdownTextSplitter

split_text = text_chunker.markdown_textsplitter(text)
file_name = "MarkdownTextSplitter_chunks.docx"
file_path = "/content/drive/MyDrive/chunking/"
heading = "MarkdownTextSplitter_chunks"

#text_chunker.save_as_word_doc(split_text, file_path + file_name ,heading)
# for i, text_chunk in enumerate(split_text):
#     print("########################################################################")
#     print(f"Chunk {i + 1}:\n{text_chunk}\n")



In [ ]:
#SentenceTransformersTokenTextSplitter chunking
split_text = text_chunker.char_count_chunking_with_custom_delimiter( text, chunk_size=200, chunk_overlap=50, splitter_type = "SentenceTransformersTokenTextSplitter")

file_name = "SentenceTransformersTokenTextSplitter.docx"
file_path = "/content/drive/MyDrive/chunking/"
heading = "SentenceTransformersTokenTextSplitter"

text_chunker.save_as_word_doc(split_text, file_path + file_name ,heading)
# for i, text_chunk in enumerate(split_text):
#     print("########################################################################")
#     print(f"Chunk {i + 1}:\n{text_chunk}\n")


Output saved as doc successfully


In [9]:
#MarkdownTextSplitter with SentenceTransformersTokenTextSplitter chunking

#file_name = "MarkdownTextSplitter_with_SentenceTransformersTokenTextSplitter_chunking.docx"
#file_path = "/content/drive/MyDrive/chunking/"
#heading = "MarkdownTextSplitter with SentenceTransformersTokenTextSplitter chunking"

markdown_split_text = text_chunker.markdown_textsplitter(text)
text_splitter = SentenceTransformersTokenTextSplitter(chunk_size=200, chunk_overlap=30)

SentenceTransformers_chunks = text_chunker.char_count_chunking_with_custom_delimiter(markdown_split_text, chunk_size=200, chunk_overlap=50, splitter_type = "SentenceTransformersTokenTextSplitter")

#text_chunker.save_as_word_doc(SentenceTransformers_split_text, file_path + file_name ,heading)

# for i, text_chunk in enumerate(SentenceTransformers_split_text):
#     print("########################################################################")
#     print(f"Chunk {i + 1}:\n{text_chunk}\n")

In [4]:
markdown_split_text = text_chunker.markdown_textsplitter(text)

SentenceTransformers_chunks = text_chunker.char_count_chunking_with_custom_delimiter(markdown_split_text, chunk_size=200, chunk_overlap=50, splitter_type = "SentenceTransformersTokenTextSplitter")


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
len(SentenceTransformers_chunks)

77

In [11]:
SentenceTransformers_chunks[0]

'mathematical foundations of data sciences gabriel peyr´ e cnrs & dma ´ecole normale sup´ erieure gabriel. peyre @ ens. fr https : / / mathematical - tours. github. io www. numerical - tours. com august 14, 2019 2 chapter 1 optimal transport 1. 1 radon measures measures. we will interchangeably the term histogram or probability vector for any element a∈σnthat belongs to the probability simplex σndef. = { a∈rn + ; [UNK] i = 1ai = 1 }. a discrete measure with weights aand locations x1,..., xn∈x reads α = [UNK] i = 1aiδxi ( 1. 1 ) whereδxis the dirac at position x, intuitively a unit of mass which is inﬁnitely concentrated at location x. such as measure describes a probability measure if, additionally, a∈σn, and more generally a positive measure if each of the “ weights ” described in vector ais positive itself. remark 1 ( general measures ). a convenient feature of ot is that it can deal with discrete and continuous “ objects ” within the same framework. such objects only need to be mode

In [1]:
#!pip install -q datasets
from datasets import Dataset

import datasets

ds = datasets.load_dataset("m-ric/huggingface_doc", split="train")

ModuleNotFoundError: No module named 'datasets'

In [15]:
ds

Dataset({
    features: ['text', 'source'],
    num_rows: 2647
})